In [3]:
import warnings
warnings.filterwarnings("ignore")
import re
import os
import numpy as np
import pandas as pd
from flask import Flask,request, render_template
import string
import nltk
from sklearn.preprocessing import maxabs_scale
from nltk.corpus import words
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
import tensorflow as tf
from keras.models import load_model

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\mnssu\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mnssu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mnssu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
ANN = tf.keras.models.load_model('ann.h5', compile = False)
RNN = tf.keras.models.load_model('rnn.h5', compile = False)
DRNN = tf.keras.models.load_model('drnn.h5', compile = False)


app = Flask(__name__, template_folder='templates')  #flask application

def decontracted(phrase):    # This function decontract words like it's to it is.    
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

def cleanPunctuations(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned
        


@app.route('/')
def hello_world():
    return render_template("home.html")


@app.route('/result', methods = ['GET', 'POST'])
def result():
    if request.method == 'POST':
        review = request.form['review']
        word = sum([len(title.split()) for title in review])
        upper= sum([sum(char.isupper() for char in title) for title in review])
        special = sum([sum(char in string.punctuation for char in title) for title in review]   )
        Uniquewords = len(set(review.split()))
        sa = SentimentIntensityAnalyzer()
        compound = sa.polarity_scores(review)['compound']
        if compound > 0:
            sentiment=1
        else:
            sentiment=0
        
        i=0
        str1=' '
        final_string=[]
        s=''
        stop = set(stopwords.words('english')) #set of stopwords
        sno = nltk.stem.SnowballStemmer('english') #initialising the snowball stemmer
        filtered_sentence=[]
        sent = decontracted(review)
        for w in sent.split():
            for cleaned_words in cleanPunctuations(w).split():
                if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                    if(cleaned_words.lower() not in stop):
                        s=(sno.stem(cleaned_words.lower())).encode('utf8')
                        filtered_sentence.append(s)
                    else:
                        continue
                else:
                    continue 

        str1 = b" ".join(filtered_sentence) #final string of cleaned words
        final_string.append(str1)
        i+=1
            
        tf_idf_vect = TfidfVectorizer(ngram_range=(1,2), min_df = 0.5, max_df = 1.0, max_features=2) #in scikit-learn
        tf_idf_vect.fit(final_string)    #fitting vectorizer on train data
        final_review = tf_idf_vect.transform(final_string).toarray()
 
        pred_ann = ANN.predict(final_review)
        pred_rnn = RNN.predict(final_review)
        pred_drnn = DRNN.predict(final_review)
        
        resultSet = pd.DataFrame()
        resultSet=resultSet.append([[ "ANN", pred_ann]])
        resultSet=resultSet.append([["RNN", pred_rnn]])
        resultSet=resultSet.append([["DRNN", pred_drnn]])
        resultSet.columns=["Models",'Predictions']
    
        return render_template("result.html", tables = [resultSet.to_html(classes='data')],tablestitle = resultSet.columns.values)


    
if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Jun/2022 09:00:32] "GET / HTTP/1.1" 200 -


127.0.0.1 - - [09/Jun/2022 09:00:37] "POST /result HTTP/1.1" 200 -
127.0.0.1 - - [09/Jun/2022 09:35:53] "POST /result HTTP/1.1" 200 -
